# Part II — Core Django  
## 10. Admin Site (Productivity Power Tool)

Django Admin is one of Django’s biggest reasons it’s used in industry. It can turn a
new project into a working internal management system in hours—but only if you
configure it properly.

In this chapter you will:

- register models (`Article`, `Tag`) in admin
- customize list pages for productivity (search, filters, ordering, pagination)
- optimize admin performance (avoid N+1 queries)
- add inline editing for relationships
- add custom actions (bulk publish/archive)
- add admin form tweaks (prepopulated slug, readonly fields)
- understand admin security and operational best practices

---

## 10.0 Learning Outcomes

By the end, you should be able to:

1. Create an admin user and log in.
2. Register models in admin and understand what you get by default.
3. Customize model admin:
   - `list_display`, `list_filter`, `search_fields`, `ordering`, `list_per_page`
   - `prepopulated_fields`, `readonly_fields`, `date_hierarchy`
   - `fieldsets`, `autocomplete_fields`, `raw_id_fields`
4. Use `TabularInline`/`StackedInline` for editing related objects.
5. Add custom actions and explain how permissions apply.
6. Optimize admin queries using `get_queryset()` + `select_related/prefetch_related`.
7. Lock down admin access and understand audit concerns.

---

## 10.1 Enable Admin (What’s Already There)

Django admin is included by default via:

- `django.contrib.admin` in `INSTALLED_APPS`
- `path("admin/", admin.site.urls)` in `config/urls.py`

You already have both.

---

## 10.2 Create a Superuser (Admin Login)

Run:

```bash
python manage.py createsuperuser
```

Enter:
- username
- email (optional)
- password

Start server:

```bash
python manage.py runserver
```

Visit:

- `http://127.0.0.1:8000/admin/`

Log in with the superuser.

### What “superuser” means
A superuser bypasses normal permission checks in admin. In production:
- superusers should be rare
- use staff users with specific permissions when possible

---

## 10.3 Register Your Models (Baseline)

Open `articles/admin.py` and register `Tag` and `Article`.

```python
from django.contrib import admin

from .models import Article, Tag


@admin.register(Tag)
class TagAdmin(admin.ModelAdmin):
    pass


@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    pass
```

Refresh admin. You’ll see Articles and Tags.

### What you get by default (and why it’s not enough)
Default admin:
- shows a basic list display (usually `__str__`)
- basic add/edit forms
- minimal search/filtering
- no optimization

For small data, fine. For real usage, you must customize.

---

## 10.4 Make Admin Useful: `list_display`, `search_fields`, `list_filter`

### 10.4.1 Improve Tag admin
Edit `articles/admin.py`:

```python
from django.contrib import admin

from .models import Article, Tag


@admin.register(Tag)
class TagAdmin(admin.ModelAdmin):
    list_display = ("id", "name", "slug")
    search_fields = ("name", "slug")
    ordering = ("name",)
    list_per_page = 50
```

#### Explanation of each option

- `list_display`: columns shown on the list page.
  - Add `id` for quick reference (helpful in debugging).
  - Show `name` and `slug` since they’re key identifiers.
- `search_fields`: enables the search bar.
  - Searching tags by name/slug is common.
  - Django uses `LIKE`/`icontains`-like queries depending on backend.
- `ordering`: default ordering of list view.
- `list_per_page`: controls pagination in admin list. Too high = slow pages; too low
  = excessive clicking.

### 10.4.2 Improve Article admin
Update `articles/admin.py`:

```python
@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    list_display = (
        "id",
        "title",
        "slug",
        "status",
        "published_at",
        "created_at",
        "updated_at",
    )
    list_filter = ("status", "created_at", "published_at", "tags")
    search_fields = ("title", "slug", "body")
    ordering = ("-created_at",)
    date_hierarchy = "created_at"
    list_per_page = 50
```

#### Explanation: what each feature does in admin UX

- `list_display`: gives editors a spreadsheet-like overview.
- `list_filter`: adds right sidebar filters.
  - Filtering by status is crucial in content workflows.
  - Filtering by tags helps with content classification.
  - Date filters speed up finding recent content.
- `search_fields`: full admin search.
  - Searching body text can be slower in large datasets—fine for now; in production
    you might limit it or use PostgreSQL full-text search later.
- `date_hierarchy`: adds “drill-down” navigation by date (year → month → day).
- `ordering`: ensures predictable list sorting.

---

## 10.5 Better Editing: `prepopulated_fields`, `readonly_fields`, `fieldsets`

### 10.5.1 Auto-fill slug from title
If slugs are manual, editors make mistakes. Auto-populating helps.

```python
@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    prepopulated_fields = {"slug": ("title",)}
    # ... keep previous options too
```

#### Why this is standard
- reduces errors
- speeds up workflow
- still allows editing slug if needed

### 10.5.2 Make timestamps read-only
These fields are system-managed. Editors should not change them.

```python
@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    readonly_fields = ("created_at", "updated_at")
```

### 10.5.3 Use `fieldsets` to organize the edit form (professional UX)
```python
@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    fieldsets = (
        ("Content", {"fields": ("title", "slug", "body")}),
        ("Publishing", {"fields": ("status", "published_at", "tags")}),
        ("Timestamps", {"fields": ("created_at", "updated_at")}),
    )
```

This makes admin forms less overwhelming as models grow.

---

## 10.6 Many-to-Many Editing UX (Tags)

By default, ManyToMany fields show a multi-select box. That’s okay for few tags, but
becomes painful with many tags.

### 10.6.1 Use filter widgets: `filter_horizontal` or `filter_vertical`
```python
@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    filter_horizontal = ("tags",)
```

This gives a nicer UI for selecting tags.

### 10.6.2 Autocomplete fields (scales better)
If you have thousands of tags, use autocomplete.

```python
@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    autocomplete_fields = ("tags",)
```

But autocomplete requires TagAdmin to define search_fields:

```python
@admin.register(Tag)
class TagAdmin(admin.ModelAdmin):
    search_fields = ("name", "slug")
```

#### Why autocomplete is better at scale
- avoids loading huge dropdowns
- queries on demand
- improves performance

---

## 10.7 Custom Columns and Computed Fields (Admin Power)

You can add computed columns in `list_display`.

### 10.7.1 Show tag count per article
```python
from django.db.models import Count

@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    list_display = ("id", "title", "status", "tag_count", "created_at")
    # (keep other settings too)

    def get_queryset(self, request):
        qs = super().get_queryset(request)
        return qs.annotate(_tag_count=Count("tags", distinct=True))

    @admin.display(ordering="_tag_count", description="Tags")
    def tag_count(self, obj):
        return obj._tag_count
```

#### Full explanation (this is important)
- `tag_count` is not a real DB column. We want it in list display.
- If we computed it per row using `obj.tags.count()`, that would cause N+1 queries.
- Instead, we annotate `_tag_count` in the queryset with one SQL query.
- Then `tag_count(self, obj)` reads `obj._tag_count` with no additional DB queries.

This is exactly the kind of optimization professionals do to keep admin fast.

---

## 10.8 Avoid N+1 Queries in Admin (Critical at Scale)

Admin list pages can easily become slow when displaying related fields.

### 10.8.1 Typical N+1 bug
If you display a related object in list_display and Django accesses it per row, it
can hit the DB repeatedly.

For ManyToMany tags, if you want to display tags as a comma-separated string, you
must prefetch.

### 10.8.2 Display tags as a string (and prefetch safely)
```python
@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    list_display = ("id", "title", "status", "tag_list", "created_at")
    filter_horizontal = ("tags",)

    def get_queryset(self, request):
        qs = super().get_queryset(request)
        return qs.prefetch_related("tags")

    @admin.display(description="Tags")
    def tag_list(self, obj):
        return ", ".join(t.slug for t in obj.tags.all())
```

#### Why prefetch is required
Without `prefetch_related("tags")`, `obj.tags.all()` would likely query per row.

With prefetch:
- one query for articles
- one query for tags for all those articles
Admin stays fast.

---

## 10.9 Admin Actions (Bulk Operations Done Right)

Actions allow selecting rows and performing operations.

### 10.9.1 Build “Publish selected” and “Archive selected”
Add to `articles/admin.py`:

```python
from django.utils import timezone


@admin.action(description="Publish selected articles")
def publish_selected(modeladmin, request, queryset):
    now = timezone.now()
    queryset.update(status=Article.Status.PUBLISHED, published_at=now)


@admin.action(description="Archive selected articles")
def archive_selected(modeladmin, request, queryset):
    queryset.update(status=Article.Status.ARCHIVED)
```

Attach actions:

```python
@admin.register(Article)
class ArticleAdmin(admin.ModelAdmin):
    actions = [publish_selected, archive_selected]
```

#### Explanation and warnings
- `queryset.update(...)` runs one SQL update—fast.
- It bypasses `save()` and signals—good for performance, but:
  - if you have side effects in `save()`, they won’t run
  - if you need per-object logic, you must iterate (slower but correct)

**Industry rule:** actions should be:
- safe
- permission-aware
- ideally reversible or auditable

---

## 10.10 Admin Permissions and Security (Professional Concerns)

### 10.10.1 Who should access admin?
Admin is not a public interface. In production:
- limit access to staff
- consider IP allowlists/VPN for internal use
- enable strong passwords and 2FA (often via third-party packages or SSO)

### 10.10.2 Staff vs superuser
- `is_staff=True` allows login to admin (with permissions)
- `is_superuser=True` bypasses permission checks

Best practice:
- grant staff users only the permissions they need:
  - “Can add article”
  - “Can change article”
  - “Can delete article”
- keep superusers limited (ops/admin team only)

### 10.10.3 Admin auditability
In serious apps, you often want:
- “who changed what and when”
- change history beyond Django’s basic admin log
- potentially external audit logs

We’ll discuss advanced audit logging later, but the mindset starts here.

---

## 10.11 Admin Performance: Pagination, Search, and Indexing

### 10.11.1 Why admin can get slow
- searching large text fields (body)
- sorting without indexes
- filtering across relationships
- displaying computed columns without annotation/prefetch

### 10.11.2 Practical guidelines
- keep `list_display` focused
- prefer indexed fields for ordering/filters
- prefetch/select_related when displaying related info
- keep `search_fields` reasonable (avoid searching huge text at scale unless you
  add proper search infrastructure)

---

# 10.12 LAB: Make Your Admin Production-Quality (Step-by-Step)

## Step 1 — Register models with useful display/search/filter
Apply the configs above.

## Step 2 — Add slug auto-population and fieldsets
Add `prepopulated_fields`, `readonly_fields`, `fieldsets`.

## Step 3 — Add tag UI improvements
Choose one:
- `filter_horizontal = ("tags",)` (simple)
- or `autocomplete_fields = ("tags",)` (scales better)

## Step 4 — Add custom actions
Add `publish_selected` and `archive_selected`.

## Step 5 — Add query optimization
- annotate tag count
- prefetch tags if you display them

## Step 6 — Verify behavior
- Create tags, create articles
- Use filters/search
- Run bulk publish/archive
- Confirm the list remains fast

---

## 10.13 Testing Admin (What You Can and Should Test)

You generally don’t unit test Django admin heavily, but there are two valuable tests:

1. **Smoke test**: admin pages load for staff users.
2. **Action tests**: custom actions do correct updates.

### 10.13.1 Admin action test example
Create `articles/tests_admin.py`:

```python
from django.contrib.auth import get_user_model
from django.test import TestCase
from django.urls import reverse
from django.utils import timezone

from articles.models import Article


class AdminActionTests(TestCase):
    def setUp(self):
        User = get_user_model()
        self.admin_user = User.objects.create_superuser(
            username="admin",
            email="admin@example.com",
            password="pass12345",
        )
        self.client.force_login(self.admin_user)

        self.a1 = Article.objects.create(
            title="A1",
            slug="a1",
            body="Body",
            status=Article.Status.DRAFT,
        )

    def test_publish_action(self):
        url = reverse("admin:articles_article_changelist")

        response = self.client.post(
            url,
            {
                "action": "publish_selected_articles",
                "_selected_action": [self.a1.id],
            },
            follow=True,
        )
        self.assertEqual(response.status_code, 200)

        self.a1.refresh_from_db()
        self.assertEqual(self.a1.status, Article.Status.PUBLISHED)
        self.assertIsNotNone(self.a1.published_at)
```

This requires that your action has a predictable name. Django derives action
function names. If you use the decorator with `@admin.action`, the action name is
the function name by default.

If your function is named `publish_selected`, your action key is `"publish_selected"`.

So update the test payload to:

```python
"action": "publish_selected",
```

And then it will work.

#### Why `force_login` is used
- bypasses login form
- ensures you’re authenticated as admin
- speeds up tests

---

## 10.14 Common Admin Pitfalls (And Fixes)

### Pitfall A: Admin pages become slow over time
Fix:
- avoid computed per-row queries
- use annotation/prefetch
- reduce list_display complexity
- add indexes for frequent filters/orderings

### Pitfall B: Slug collisions
Even with `prepopulated_fields`, editors can produce duplicates.
Fix:
- keep `unique=True` (you already have)
- show helpful validation errors
- consider auto-generating unique slugs in save logic later

### Pitfall C: Unsafe actions
Actions can mass-edit thousands of rows.
Fix:
- confirm actions with careful UI and permission checks
- consider limiting actions or requiring additional confirmation for destructive actions

---

## 10.15 Exercises (Do These Before Proceeding)

1. Add a computed column:
   - “Is Published?” boolean display based on status.
   - Use `@admin.display(boolean=True)`.

2. Add a filter:
   - filter by `tags` (already shown) and confirm it works.

3. Optimize query count:
   - If you display tags, ensure you prefetch.
   - Add a small test with `assertNumQueries` around the admin changelist view
     (advanced; optional).

4. Add a custom action “Mark as Draft” that clears `published_at`.

5. Create a staff user (not superuser) and grant only:
   - change article
   - view article
   Confirm they can edit but not delete.

---

## 10.16 Chapter Summary

- Django admin is a production-grade internal tool when configured well.
- Customize list pages for productivity (display/search/filter/ordering).
- Use prepopulated fields and fieldsets for better editing UX.
- Optimize admin to avoid N+1 queries using annotate/prefetch/select_related.
- Admin actions can be powerful—make them safe and auditable.
- Control admin access strictly in production.

---

Next chapter: **Part II — Chapter 11: Forms (HTML Forms, Validation, UX)**  
We’ll build real create/edit pages for articles with Django Forms and ModelForms,
including file upload basics, validation patterns, and user feedback with messages.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='9. django_orm.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='11. forms.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
